# ExperimentLoader + MLflow Smoke Test

This notebook demonstrates loading a run's config, metrics, and model using `ExperimentLoader` and the new MLflow reader.

Prerequisites:
- Access to the MLflow tracking server (Databricks).
- Local environment has Databricks credentials available (host/token), or you pass `tracking_uri="databricks"`.

You can change `run_id` below to any run you want to inspect.

In [1]:
# --- Configuration ---
run_id = "138baafac5a34726ab070bb1be72a83d"  # Replace if desired
tracking_uri = "databricks"  # Use 'databricks' or None to rely on env
print(f'Using run_id={run_id}, tracking_uri={tracking_uri}')

Using run_id=138baafac5a34726ab070bb1be72a83d, tracking_uri=databricks


In [2]:
# --- Import the loader ---
from simplexity.loaders import ExperimentLoader

loader = ExperimentLoader.from_mlflow(run_id, tracking_uri=tracking_uri)
print('Loader created.')

Loader created.


In [3]:
# --- Load and inspect the run config ---
cfg = loader.load_config()
print('Predictive model target:', cfg.predictive_model.instance._target_)
print('Has persistence:', bool(cfg.persistence))
if cfg.persistence:
    print('Persister target:', cfg.persistence.instance._target_)
cfg

Predictive model target: transformer_lens.HookedTransformer
Has persistence: True
Persister target: simplexity.persistence.s3_persister.S3Persister.from_config


{'seed': 42, 'device': 'auto', 'experiment_name': 'example_basic_mess3', 'run_name': '2025-09-09_22-09-10_example_basic_mess3_42', 'training_data_generator': {'name': 'mess3_085', 'vocab_size': 4, 'instance': {'_target_': 'simplexity.generative_processes.builder.build_hidden_markov_model', 'process_name': 'mess3', 'x': 0.05, 'a': 0.85}, 'bos_token': 3, 'eos_token': None}, 'validation_data_generator': {'name': 'mess3_085', 'vocab_size': 4, 'instance': {'_target_': 'simplexity.generative_processes.builder.build_hidden_markov_model', 'process_name': 'mess3', 'x': 0.05, 'a': 0.85}, 'bos_token': 3, 'eos_token': None}, 'predictive_model': {'name': 'transformer_lens_2L4H', 'instance': {'_target_': 'transformer_lens.HookedTransformer', 'cfg': {'_target_': 'transformer_lens.HookedTransformerConfig', 'n_layers': 2, 'n_heads': 4, 'd_model': 64, 'd_head': 16, 'n_ctx': 6, 'd_vocab': 4, 'act_fn': 'relu', 'normalization_type': 'LN', 'positional_embedding_type': 'standard', 'attn_only': False, 'seed':

In [4]:
# --- Checkpoints: list and load latest model ---
steps = loader.list_checkpoints()
print('Checkpoints:', steps)
latest = loader.latest_checkpoint()
print('Latest checkpoint:', latest)

try:
    model = loader.load_model(step="latest")
    print('Model loaded successfully:', type(model))
except Exception as e:
    print('Could not load model:', e)


InstantiationException: Error locating target 'transformer_lens.HookedTransformer', set env var HYDRA_FULL_ERROR=1 to see chained exception.
full_key: predictive_model.instance

In [ ]:
# --- Load metrics (all) and a filtered view (e.g., validation/*) ---
import pandas as pd

df_all = loader.load_metrics()
display(df_all.head())

df_val = loader.load_metrics(pattern="validation/*")
display(df_val.head())
print(f'Total metrics rows: all={len(df_all)}, validation={len(df_val)}')


In [ ]:
# --- Quick plot (optional) ---
import matplotlib.pyplot as plt

if not df_val.empty:
    for metric in sorted(df_val['metric'].unique())[:3]:
        sub = df_val[df_val['metric'] == metric]
        plt.plot(sub['step'], sub['value'], label=metric)
    plt.xlabel('step')
    plt.ylabel('value')
    plt.title('Validation metrics (subset)')
    plt.legend()
    plt.show()
else:
    print('No validation/* metrics to plot.')


## Notes
- `ExperimentLoader` reconstructs the model and persister from the saved Hydra `config.yaml` artifact.
- Checkpoint discovery is provided by persisters (`list_checkpoints`, `latest_checkpoint`).
- If the run has no persistence configured or no checkpoints, loading the model will raise an error (caught above).
- You can use glob-style metric filters, e.g., `validation/*`.